<a href="https://colab.research.google.com/github/Herouala/ndn-caching-alfu-cd/blob/main/ALFU_CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict, Counter, deque
from scipy.stats import chi2_contingency
from matplotlib.backends.backend_pdf import PdfPages

# Base Cache Strategy class
class CacheStrategy:
    def __init__(self, size):
        self.size = size
        self.cache = {}
        self.hits = 0
        self.misses = 0
        self.evictions = 0
        self.occupancy = []  # Track cache utilization over time
        self.access_freq = Counter()  # Track frequencies for fairness

    def access(self, item):
        raise NotImplementedError

    def reset_stats(self):
        self.hits = 0
        self.misses = 0
        self.evictions = 0
        self.occupancy = []
        self.access_freq = Counter()

    def record_metrics(self):
        # Cache utilization (occupancy)
        self.occupancy.append(len(self.cache) / self.size if self.size > 0 else 0)
        # Update frequency of cached items
        for k in self.cache:
            self.access_freq[k] += 1

    def get_metrics(self, n_requests):
        # Utilization: average occupancy over time
        utilization = np.mean(self.occupancy) if self.occupancy else 0

        # Fairness index: Jain’s index over cached items frequencies
        if len(self.access_freq) > 0:
            x = np.array(list(self.access_freq.values()))
            fairness = (np.sum(x) ** 2) / (len(x) * np.sum(x ** 2)) if np.sum(x ** 2) > 0 else 0
        else:
            fairness = 0



        return utilization, fairness


# FIFO Cache
class FIFO(CacheStrategy):
    def __init__(self, size):
        super().__init__(size)
        self.order = deque()

    def access(self, item):
        if item in self.cache:
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                evicted = self.order.popleft()
                del self.cache[evicted]
                self.evictions += 1
            self.cache[item] = True
            self.order.append(item)
        self.record_metrics()

# LRU Cache
class LRU(CacheStrategy):
    def __init__(self, size):
        super().__init__(size)
        self.cache = OrderedDict()

    def access(self, item):
        if item in self.cache:
            self.cache.move_to_end(item)
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                self.cache.popitem(last=False)
                self.evictions += 1
            self.cache[item] = True
        self.record_metrics()

# LFU Cache
class LFU(CacheStrategy):
    def __init__(self, size):
        super().__init__(size)
        self.freq = Counter()

    def access(self, item):
        if item in self.cache:
            self.freq[item] += 1
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                min_items = [k for k in self.cache if self.freq[k] == min(self.freq.values())]
                evicted = min_items[0]
                del self.cache[evicted]
                del self.freq[evicted]
                self.evictions += 1
            self.cache[item] = True
            self.freq[item] = 1
        self.record_metrics()

# LFUDA
class LFUDA(CacheStrategy):
    def __init__(self, size):
        super().__init__(size)
        self.freq = Counter()
        self.age = 0

    def access(self, item):
        if item in self.cache:
            self.freq[item] += 1
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                self.age += 1
                eff_freq = {k: self.freq[k] + self.age for k in self.cache}
                evicted = min(eff_freq, key=eff_freq.get)
                del self.cache[evicted]
                del self.freq[evicted]
                self.evictions += 1
            self.cache[item] = True
            self.freq[item] = 0
        self.record_metrics()

# SIEVE Cache
class SIEVE(CacheStrategy):
    def __init__(self, size):
        super().__init__(size)
        self.queue = deque()
        self.accessed = {}

    def access(self, item):
        if item in self.cache:
            self.accessed[item] = True
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                while True:
                    candidate = self.queue[0]
                    if self.accessed[candidate]:
                        self.accessed[candidate] = False
                        self.queue.rotate(-1)
                    else:
                        evicted = self.queue.popleft()
                        del self.cache[evicted]
                        del self.accessed[evicted]
                        self.evictions += 1
                        break
            self.cache[item] = True
            self.accessed[item] = False
            self.queue.append(item)
        self.record_metrics()

# ALFU-CD (renamed from ADWIN-LFU)
class ALFU_CD(CacheStrategy):
    def __init__(self, size, w2_size=1000, alpha=0.7, p_threshold=0.05):
        super().__init__(size)
        self.w1_freq = Counter()
        self.w2 = deque(maxlen=w2_size)
        self.alpha = alpha
        self.p_threshold = p_threshold
        self.change_detected = False

    def _detect_change(self):
        if len(self.w2) < self.w2.maxlen:
            return False
        half = len(self.w2) // 2
        freq1 = Counter(list(self.w2)[:half])
        freq2 = Counter(list(self.w2)[half:])
        keys = set(freq1) | set(freq2)
        if not keys:
            return False
        obs = np.array([[freq1.get(k, 0), freq2.get(k, 0)] for k in keys])
        try:
            _, p, _, _ = chi2_contingency(obs)
            return p < self.p_threshold
        except ValueError:
            return False

    def access(self, item):
        self.w2.append(item)
        if item in self.cache:
            self.w1_freq[item] += 1
            self.hits += 1
        else:
            self.misses += 1
            if len(self.cache) >= self.size:
                if self.change_detected:
                    recent_freq = Counter(self.w2)
                    cache_freq = {k: recent_freq.get(k, 0) for k in self.cache}
                else:
                    cache_freq = {k: self.w1_freq[k] for k in self.cache}
                min_items = [k for k, v in cache_freq.items() if v == min(cache_freq.values())]
                evicted = min_items[0]
                del self.cache[evicted]
                self.w1_freq.pop(evicted, None)
                self.evictions += 1
            self.cache[item] = True
            self.w1_freq[item] += 1
        if self._detect_change():
            self.change_detected = True
            freq_w2 = Counter(self.w2)
            for k in freq_w2:
                self.w1_freq[k] = (1 - self.alpha) * self.w1_freq.get(k, 0) + self.alpha * freq_w2[k]
        else:
            self.change_detected = False
        self.record_metrics()

import numpy as np

# Generate requests with Zipf distribution and three phases of alpha
def generate_requests_with_change(n_requests, n_items, alpha1, alpha2, alpha3):
    # Split requests into three roughly equal parts
    part1 = n_requests // 3
    part2 = n_requests // 3
    part3 = n_requests - part1 - part2  # ensures total = n_requests

    # Phase 1
    items1 = np.arange(1, n_items + 1)
    probs1 = items1 ** (-alpha1)
    probs1 /= probs1.sum()
    req1 = np.random.choice(items1, part1, p=probs1)

    # Phase 2
    items2 = np.arange(1, n_items + 1)
    probs2 = items2 ** (-alpha2)
    probs2 /= probs2.sum()
    req2 = np.random.choice(items2, part2, p=probs2)

    # Phase 3
    items3 = np.arange(1, n_items + 1)
    probs3 = items3 ** (-alpha3)
    probs3 /= probs3.sum()
    req3 = np.random.choice(items3, part3, p=probs3)

    return np.concatenate((req1, req2, req3))


# Simulation function
def simulate(strategy_class, requests, cache_size):
    strategy = strategy_class(cache_size)
    for item in requests:
        strategy.access(item)
    hit_rate = strategy.hits / len(requests) if len(requests) > 0 else 0
    evictions = strategy.evictions/ len(requests) if len(requests) > 0 else 0
    utilization, fairness= strategy.get_metrics(len(requests))
    return hit_rate, evictions, utilization, fairness

# Main simulation and plotting
def run_simulations():
    n_requests = 10000
    n_items = 1000
    cache_sizes = list(range(10, 101, 10))
    num_runs = 10  # nombre de répétitions

    strategies = {
        'FIFO': FIFO,
        'LRU': LRU,
        'LFU': LFU,
        'LFUDA': LFUDA,
        'SIEVE': SIEVE,
        'ALFU-CD': ALFU_CD
    }

    # Stockage des résultats multi-runs
    metrics = {
        'Hit Rate': {s: [] for s in strategies},
        'Evictions': {s: [] for s in strategies},
        'Utilization': {s: [] for s in strategies},
        'Fairness': {s: [] for s in strategies},

    }

    fixed_alpha1, fixed_alpha2, fixed_alpha3 = 0.5, 1.0, 1.5

    for size in cache_sizes:
        for name, cls in strategies.items():
            run_results = {'hr': [], 'ev': [], 'ut': [], 'fair': []}
            for _ in range(num_runs):
                requests = generate_requests_with_change(n_requests, n_items, fixed_alpha1, fixed_alpha2, fixed_alpha3)
                hr, ev, ut, fair = simulate(cls, requests, size)
                run_results['hr'].append(hr)
                run_results['ev'].append(ev)
                run_results['ut'].append(ut)
                run_results['fair'].append(fair)


            # Moyenne sur num_runs
            metrics['Hit Rate'][name].append(np.mean(run_results['hr']))
            metrics['Evictions'][name].append(np.mean(run_results['ev']))
            metrics['Utilization'][name].append(np.mean(run_results['ut']))
            metrics['Fairness'][name].append(np.mean(run_results['fair']))


    # Styles
    line_styles = ['-', '--', '-.', ':', '-', '--']
    markers = ['o', 's', '^', 'D', 'x', '*']

    # Export PDF
    pdf_filename = "cache_strategies_results.pdf"
    with PdfPages(pdf_filename) as pdf:
        for metric_name, metric_dict in metrics.items():
            plt.figure(figsize=(10, 6))
            for i, (name, vals) in enumerate(metric_dict.items()):
                plt.plot(cache_sizes, vals, label=name,
                         marker=markers[i % len(markers)],
                         linestyle=line_styles[i % len(line_styles)])
            plt.xlabel('Cache Size')
            plt.ylabel(metric_name)
            plt.title(f'Average {metric_name} vs Cache Size ')
            plt.xticks(cache_sizes)  # 🔹 Forcer affichage de toutes les tailles de cache
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()  # Sauvegarde dans le PDF
            plt.close()

    print(f"✅ Résultats exportés dans {pdf_filename}")

def run_w2_analysis():
    n_requests = 10000
    n_items = 1000
    cache_size = 100   # Fix cache size for W2 study
    num_runs = 100     # repeat runs for averaging
    fixed_alpha1, fixed_alpha2, fixed_alpha3 = 0.5, 1.0, 1.5

    # W2 sizes to test
    w2_sizes = list(range(100, 1001, 100))

    # Collect metrics
    metrics_w2 = {
        'Hit Rate': [],
        'Evictions': [],
        'Utilization': [],
        'Fairness': []
    }

    for w2 in w2_sizes:
        run_results = {'hr': [], 'ev': [], 'ut': [], 'fair': []}
        for _ in range(num_runs):
            requests = generate_requests_with_change(n_requests, n_items,
                                                     fixed_alpha1, fixed_alpha2, fixed_alpha3)
            strategy = ALFU_CD(cache_size, w2_size=w2, alpha=0.7, p_threshold=0.05)
            for item in requests:
                strategy.access(item)
            hr = strategy.hits / len(requests)
            ev = strategy.evictions / len(requests)
            ut, fair = strategy.get_metrics(len(requests))
            run_results['hr'].append(hr)
            run_results['ev'].append(ev)
            run_results['ut'].append(ut)
            run_results['fair'].append(fair)

        # Average per W2
        metrics_w2['Hit Rate'].append(np.mean(run_results['hr']))
        metrics_w2['Evictions'].append(np.mean(run_results['ev']))
        metrics_w2['Utilization'].append(np.mean(run_results['ut']))
        metrics_w2['Fairness'].append(np.mean(run_results['fair']))

    # Export PDF
    pdf_filename = "w2_analysis_results.pdf"
    with PdfPages(pdf_filename) as pdf:
        for metric_name, vals in metrics_w2.items():
            plt.figure(figsize=(10, 6))
            plt.plot(w2_sizes, vals, marker='*', linestyle='--',color='brown' )
            plt.xlabel('W2 Window Size')
            plt.ylabel(metric_name)
            plt.title(f'{metric_name} vs W2 Window Size (ALFU-CD)')
            plt.xticks(w2_sizes)  # 🔹 Forcer affichage de toutes les tailles de cache
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    print(f"✅ W2 analysis results exported to {pdf_filename}")


# Run both analyses
run_w2_analysis()   # new W2 study

# Run the simulations
#run_simulations()

✅ W2 analysis results exported to w2_analysis_results.pdf
